# Notebook goal: webscrape Goodreads Booktok shelf

10/5 update: realized why Abigail had to import her Goodreads login and use all those extra packages - initial webscrape only scraped the first page 25x (yes I cried in the corner all night)

In [3]:
import requests
from requests import get 
from bs4 import BeautifulSoup as BS
import re
import pandas as pd
import numpy as np
import time
from time import sleep
from random import randint
import matplotlib.pyplot as plt
%matplotlib inline
import lxml 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service  # Import Service for Chrome
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

driver.get("https://www.goodreads.com/user/sign_in")

button = driver.find_element(By.LINK_TEXT, "Sign in with email")
button.click()

username = driver.find_element(By.NAME, "email")
username.send_keys("jennifer.toppins@gmail.com")

# driver.quit()

#headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'} #trying my user agent here instead to see if it helps
# used header here to specify that i'm not a bot but a user, that way goodreads let me in

Next Abigail imported json and keys to get her Goodreads password in...

In [6]:
import json
with open('credentials.json', 'w') as f:
    json.dump(credentials, f) #need to put my credentials in a file to use later...

with open('credentials.json') as f:
    loaded_credentials = json.load(f)

time.sleep(10) #adding some sleep in here because I got the MaxRetryError as I was practicing this 

In [8]:
email = loaded_credentials['email']
password = loaded_credentials['password']

In [10]:
password_field = driver.find_element(By.NAME, "password")
password_field.send_keys(password)

In [12]:
signin = driver.find_element(By.ID, "signInSubmit")
signin.click()

In [14]:
# URL = 'https://www.goodreads.com/shelf/show/booktok'

# # headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'}
# # used header here to specify that i'm not a bot but a user, that way goodreads let me in
# response = requests.get(URL, headers=headers)
# # print(response.content.decode())

In [16]:
# cards = soup.findAll('div', {'class': 'elementList'}) #cards contains all the elements on the page 
# print(cards)

In [18]:
# #testing

# page = 1
# while page != 5:
#       url = f"https://www.goodreads.com/shelf/show/booktok?page={page}"
#       print(url)
#       page = page + 1

In [20]:
page = 1
titles = []
authors = []
hrefs = []
shelved = []
images = []
all_meta = []
for page in range(1, 26, 1):
    url = f'https://www.goodreads.com/shelf/show/booktok?page={page}'

    #error testing the urls
    print(f'Processing URL: {url}')

    driver.get(url)
    time.sleep(5)

    #response = requests.get(url, headers=headers)
    # if response.status_code == 200:
    #     html = response.content
    all_soup = BS(driver.page_source, "lxml")

    for book in all_soup.find_all('div', {'class':'left'}): #get the all the info for all books here
        title_tag = book.find('a', class_='bookTitle') #get titles here 
        author_tag = book.find('span', {'itemprop': 'name'}) #get authors here
        urls_tag = book.find('a', attrs={'class': 'bookTitle', 'href': re.compile('^/book/show/')}) #get urls here
        shelved_tag = book.find(string=re.compile(r'shelved'))
        images_tag = book.find('a', attrs={'class':'leftAlignedImage'}) #get image srcs here
        meta_tag = book.find(string=re.compile('avg')) #get meta info here for stripping later 
        if title_tag and author_tag and urls_tag and shelved_tag and images_tag:
            titles.append(title_tag.get_text(strip=True))
            authors.append(author_tag.get_text(strip=True))
            hrefs.append(urls_tag.get('href'))
            shelved.append(shelved_tag)
            images.append(images_tag.find('img'))
            all_meta.append(meta_tag)
        
    print(f"Page {page} processed.")
    
    time.sleep(8)  # Add delay between requests to keep from timing out at page 10

string = 'https://www.goodreads.com'
urls = [string + s for s in hrefs] #finish building out the urls 
shelved_as_booktok = [re.sub("[^0-9]", "", x) for x in shelved] #clean up the numbers like above 
all_image_srcs = [x.get('src') for x in images]


Processing URL: https://www.goodreads.com/shelf/show/booktok?page=1
Page 1 processed.
Processing URL: https://www.goodreads.com/shelf/show/booktok?page=2
Page 2 processed.
Processing URL: https://www.goodreads.com/shelf/show/booktok?page=3
Page 3 processed.
Processing URL: https://www.goodreads.com/shelf/show/booktok?page=4
Page 4 processed.
Processing URL: https://www.goodreads.com/shelf/show/booktok?page=5
Page 5 processed.
Processing URL: https://www.goodreads.com/shelf/show/booktok?page=6
Page 6 processed.
Processing URL: https://www.goodreads.com/shelf/show/booktok?page=7
Page 7 processed.
Processing URL: https://www.goodreads.com/shelf/show/booktok?page=8
Page 8 processed.
Processing URL: https://www.goodreads.com/shelf/show/booktok?page=9
Page 9 processed.
Processing URL: https://www.goodreads.com/shelf/show/booktok?page=10
Page 10 processed.
Processing URL: https://www.goodreads.com/shelf/show/booktok?page=11
Page 11 processed.
Processing URL: https://www.goodreads.com/shelf/sh

In [22]:
print("Titles:")
print(titles)
print(len(titles))

Titles:
['The Seven Husbands of Evelyn Hugo (Hardcover)', 'It Ends with Us (It Ends with Us #1)', 'The Song of Achilles (Paperback)', 'The Cruel Prince (The Folk of the Air, #1)', 'A Court of Thorns and Roses (A Court of Thorns and Roses, #1)', 'The Love Hypothesis (Paperback)', 'The Invisible Life of Addie LaRue (Hardcover)', 'Six of Crows (Six of Crows, #1)', 'From Blood and Ash (Blood and Ash, #1)', 'Red, White & Royal Blue (Paperback)', "A Good Girl's Guide to Murder (A Good Girl's Guide to Murder, #1)", 'They Both Die at the End (Death-Cast, #1)', 'The Inheritance Games (The Inheritance Games, #1)', 'Fourth Wing (The Empyrean, #1)', 'Verity (ebook)', 'Beach Read (Paperback)', 'People We Meet on Vacation (Paperback)', 'Shatter Me (Shatter Me, #1)', 'The Spanish Love Deception (Spanish Love Deception, #1)', 'Ugly Love (Kindle Edition)', 'We Were Liars (Kindle Edition)', 'Twisted Love (Twisted, #1)', 'Icebreaker (UCMH, #1)', 'A Court of Mist and Fury (A Court of Thorns and Roses, #2)

In [24]:
print("\nAuthors:")
print(authors)
print(len(authors))


Authors:
['Taylor Jenkins Reid', 'Colleen Hoover', 'Madeline Miller', 'Holly Black', 'Sarah J. Maas', 'Ali Hazelwood', 'Victoria Schwab', 'Leigh Bardugo', 'Jennifer L. Armentrout', 'Casey McQuiston', 'Holly  Jackson', 'Adam Silvera', 'Jennifer Lynn Barnes', 'Rebecca Yarros', 'Colleen Hoover', 'Emily Henry', 'Emily Henry', 'Tahereh Mafi', 'Elena  Armas', 'Colleen Hoover', 'E. Lockhart', 'Ana Huang', 'Hannah  Grace', 'Sarah J. Maas', 'Stephanie Garber', 'Taylor Jenkins Reid', 'Sarah J. Maas', 'Lauren Asher', 'Leigh Bardugo', 'Emily Henry', 'Christina Lauren', 'Madeline Miller', 'Kerri Maniscalco', 'Sally Rooney', 'Scarlett St.  Clair', 'Mary E. Pearson', 'Matt Haig', 'Sally  Thorne', 'Victoria Aveyard', 'Lucy Score', 'Tessa Bailey', 'Sarah J. Maas', 'Chloe Gong', 'Penelope Douglas', 'Olivie Blake', 'Sarah J. Maas', 'H.D. Carlton', 'Elle Kennedy', 'Hanya Yanagihara', 'Sarah J. Maas', 'M.L. Rio', 'Christina Lauren', 'Shelby Mahurin', 'Samantha    Shannon', 'Donna Tartt', 'Katee Robert', '

In [26]:
print("Urls:")
print(urls)
print(len(urls))

Urls:
['https://www.goodreads.com/book/show/32620332-the-seven-husbands-of-evelyn-hugo', 'https://www.goodreads.com/book/show/27362503-it-ends-with-us', 'https://www.goodreads.com/book/show/13623848-the-song-of-achilles', 'https://www.goodreads.com/book/show/26032825-the-cruel-prince', 'https://www.goodreads.com/book/show/50659467-a-court-of-thorns-and-roses', 'https://www.goodreads.com/book/show/56732449-the-love-hypothesis', 'https://www.goodreads.com/book/show/50623864-the-invisible-life-of-addie-larue', 'https://www.goodreads.com/book/show/23437156-six-of-crows', 'https://www.goodreads.com/book/show/52831200-from-blood-and-ash', 'https://www.goodreads.com/book/show/41150487-red-white-royal-blue', 'https://www.goodreads.com/book/show/40916679-a-good-girl-s-guide-to-murder', 'https://www.goodreads.com/book/show/33385229-they-both-die-at-the-end', 'https://www.goodreads.com/book/show/52439531-the-inheritance-games', 'https://www.goodreads.com/book/show/61431922-fourth-wing', 'https://

In [28]:
print('Number of Times Shelved as Booktok:')
print(shelved_as_booktok)
print(len(shelved_as_booktok))

Number of Times Shelved as Booktok:
['1023', '875', '806', '783', '761', '720', '655', '611', '570', '570', '566', '543', '532', '527', '527', '523', '519', '518', '508', '480', '446', '435', '430', '426', '421', '408', '387', '385', '380', '377', '374', '372', '366', '357', '350', '348', '347', '346', '344', '338', '338', '337', '334', '323', '322', '320', '314', '313', '306', '305', '293', '291', '291', '289', '279', '276', '274', '272', '270', '268', '267', '265', '260', '259', '254', '252', '248', '246', '246', '245', '245', '244', '241', '240', '240', '238', '236', '235', '234', '231', '230', '229', '225', '218', '215', '209', '208', '203', '199', '197', '197', '193', '192', '189', '188', '186', '185', '185', '183', '180', '179', '177', '175', '174', '173', '168', '165', '164', '164', '163', '160', '160', '160', '159', '157', '155', '154', '154', '154', '153', '153', '151', '150', '149', '146', '146', '146', '146', '145', '145', '144', '144', '144', '144', '141', '141', '141', '14

In [30]:
print("Image sources:")
print(all_image_srcs)
print(len(all_image_srcs))

Image sources:
['https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1664458703l/32620332._SY75_.jpg', 'https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1688011813l/27362503._SY75_.jpg', 'https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1357177533l/13623848._SY75_.jpg', 'https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1574535986l/26032825._SY75_.jpg', 'https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1620324329l/50659467._SY75_.jpg', 'https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1611937942l/56732449._SX50_.jpg', 'https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1584633432l/50623864._SY75_.jpg', 'https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1651710803l/23437156._SY75_.jpg', 'https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1585598504l/52831200._SX50_.jpg', 'https://i.gr-assets.com/images/S/compr

In [32]:
print("Meta data:")
print(all_meta)
print(len(all_meta))

Meta data:
['\n                avg rating 4.41 —\n                3,340,727 ratings  —\n                published 2017\n              ', '\n                avg rating 4.14 —\n                3,863,992 ratings  —\n                published 2016\n              ', '\n                avg rating 4.32 —\n                1,628,438 ratings  —\n                published 2011\n              ', '\n                avg rating 4.03 —\n                1,378,006 ratings  —\n                published 2018\n              ', '\n                avg rating 4.18 —\n                3,068,237 ratings  —\n                published 2015\n              ', '\n                avg rating 4.13 —\n                1,508,362 ratings  —\n                published 2021\n              ', '\n                avg rating 4.18 —\n                1,178,268 ratings  —\n                published 2020\n              ', '\n                avg rating 4.48 —\n                1,008,395 ratings  —\n                published 2015\n     

In [34]:
#clean the meta data again
all_meta_cleaned = [x.replace('\n', '').replace('—', '').replace('                 ', '&').replace('avg rating','').replace('ratings','').replace('published','') for x in all_meta]
#created & to use as a delimiter later when i split the columns in the df 
all_meta_cleaned

['                 4.41&3,340,727 &  2017              ',
 '                 4.14&3,863,992 &  2016              ',
 '                 4.32&1,628,438 &  2011              ',
 '                 4.03&1,378,006 &  2018              ',
 '                 4.18&3,068,237 &  2015              ',
 '                 4.13&1,508,362 &  2021              ',
 '                 4.18&1,178,268 &  2020              ',
 '                 4.48&1,008,395 &  2015              ',
 '                 4.23&648,299 &  2020              ',
 '                 4.08&1,070,667 &  2019              ',
 '                 4.32&1,316,558 &  2019              ',
 '                 3.76&785,013 &  2017              ',
 '                 4.15&825,954 &  2020              ',
 '                 4.57&2,046,307 &  2023              ',
 '                 4.31&3,018,593 &  2018              ',
 '                 4.00&1,297,599 &  2020              ',
 '                 3.86&1,288,658 &  2021              ',
 '                 3

In [36]:
all_meta_cleaned2 = [x.strip() for x in all_meta_cleaned]
all_meta_cleaned2

['4.41&3,340,727 &  2017',
 '4.14&3,863,992 &  2016',
 '4.32&1,628,438 &  2011',
 '4.03&1,378,006 &  2018',
 '4.18&3,068,237 &  2015',
 '4.13&1,508,362 &  2021',
 '4.18&1,178,268 &  2020',
 '4.48&1,008,395 &  2015',
 '4.23&648,299 &  2020',
 '4.08&1,070,667 &  2019',
 '4.32&1,316,558 &  2019',
 '3.76&785,013 &  2017',
 '4.15&825,954 &  2020',
 '4.57&2,046,307 &  2023',
 '4.31&3,018,593 &  2018',
 '4.00&1,297,599 &  2020',
 '3.86&1,288,658 &  2021',
 '3.85&981,921 &  2011',
 '3.82&729,544 &  2021',
 '4.03&2,000,733 &  2014',
 '3.66&1,197,948 &  2014',
 '3.72&1,103,024 &  2021',
 '3.82&937,655 &  2022',
 '4.65&2,330,636 &  2016',
 '3.98&615,790 &  2016',
 '4.20&1,609,643 &  2019',
 '4.18&1,754,927 &  2012',
 '3.84&551,228 &  2021',
 '3.93&996,578 &  2012',
 '4.13&1,221,175 &  2022',
 '3.91&937,670 &  2019',
 '4.23&1,140,998 &  2018',
 '3.88&303,934 &  2020',
 '3.81&1,520,220 &  2018',
 '3.82&356,867 &  2019',
 '4.13&159,710 &  2018',
 '3.99&1,958,017 &  2020',
 '3.88&750,410 &  2016',
 '

## create df for all pages data

In [39]:
booktok_books_df = pd.DataFrame(
    {'Title': titles,
     'Author': authors,
     'Num_shelved_booktok': shelved_as_booktok,
     'Meta': all_meta_cleaned2,
     'Goodreads URL': urls,
     'Image URL': all_image_srcs
    })

In [41]:
booktok_books_df.head()

,Title,Author,Num_shelved_booktok,Meta,Goodreads URL,Image URL
0,The Seven Husbands of Evelyn Hugo (Hardcover),Taylor Jenkins Reid,1023,"4.41&3,340,727 & 2017",https://www.goodreads.com/book/show/32620332-t...,https://i.gr-assets.com/images/S/compressed.ph...
1,It Ends with Us (It Ends with Us #1),Colleen Hoover,875,"4.14&3,863,992 & 2016",https://www.goodreads.com/book/show/27362503-i...,https://i.gr-assets.com/images/S/compressed.ph...
2,The Song of Achilles (Paperback),Madeline Miller,806,"4.32&1,628,438 & 2011",https://www.goodreads.com/book/show/13623848-t...,https://i.gr-assets.com/images/S/compressed.ph...
3,"The Cruel Prince (The Folk of the Air, #1)",Holly Black,783,"4.03&1,378,006 & 2018",https://www.goodreads.com/book/show/26032825-t...,https://i.gr-assets.com/images/S/compressed.ph...
4,A Court of Thorns and Roses (A Court of Thorns...,Sarah J. Maas,761,"4.18&3,068,237 & 2015",https://www.goodreads.com/book/show/50659467-a...,https://i.gr-assets.com/images/S/compressed.ph...


In [43]:
booktok_books_df.tail()

,Title,Author,Num_shelved_booktok,Meta,Goodreads URL,Image URL
1245,"Barbarian Lover (Ice Planet Barbarians, #3)",Ruby Dixon,19,"3.84&62,683 & 2015",https://www.goodreads.com/book/show/60629205-b...,https://i.gr-assets.com/images/S/compressed.ph...
1246,When Sinners Play (Sinners of Hawthorne Univer...,Eva Ashwood,19,"4.03&8,489 & 2020",https://www.goodreads.com/book/show/61095459-w...,https://i.gr-assets.com/images/S/compressed.ph...
1247,Cultish: The Language of Fanaticism (Hardcover),Amanda Montell,19,"3.85&91,989 & 2021",https://www.goodreads.com/book/show/55338982-c...,https://i.gr-assets.com/images/S/compressed.ph...
1248,The Memory Police (Hardcover),Yōko Ogawa,19,"3.72&100,268 & 1994",https://www.goodreads.com/book/show/37004370-t...,https://i.gr-assets.com/images/S/compressed.ph...
1249,Him (Him #1),Sarina Bowen,19,"4.23&74,130 & 2015",https://www.goodreads.com/book/show/59754566-him,https://i.gr-assets.com/images/S/compressed.ph...


In [45]:
all_meta_df = booktok_books_df['Meta'].str.split(pat='&', expand=True)

In [47]:
all_meta_df.columns = ['avg rating', 'num_ratings', 'year_published']
all_meta_df.head()

,avg rating,num_ratings,year_published
0,4.41,"3,340,727",2017
1,4.14,"3,863,992",2016
2,4.32,"1,628,438",2011
3,4.03,"1,378,006",2018
4,4.18,"3,068,237",2015


In [49]:
booktok_books_df_final = booktok_books_df.join(all_meta_df)
booktok_books_df_final.head()

,Title,Author,Num_shelved_booktok,Meta,Goodreads URL,Image URL,avg rating,num_ratings,year_published
0,The Seven Husbands of Evelyn Hugo (Hardcover),Taylor Jenkins Reid,1023,"4.41&3,340,727 & 2017",https://www.goodreads.com/book/show/32620332-t...,https://i.gr-assets.com/images/S/compressed.ph...,4.41,"3,340,727",2017
1,It Ends with Us (It Ends with Us #1),Colleen Hoover,875,"4.14&3,863,992 & 2016",https://www.goodreads.com/book/show/27362503-i...,https://i.gr-assets.com/images/S/compressed.ph...,4.14,"3,863,992",2016
2,The Song of Achilles (Paperback),Madeline Miller,806,"4.32&1,628,438 & 2011",https://www.goodreads.com/book/show/13623848-t...,https://i.gr-assets.com/images/S/compressed.ph...,4.32,"1,628,438",2011
3,"The Cruel Prince (The Folk of the Air, #1)",Holly Black,783,"4.03&1,378,006 & 2018",https://www.goodreads.com/book/show/26032825-t...,https://i.gr-assets.com/images/S/compressed.ph...,4.03,"1,378,006",2018
4,A Court of Thorns and Roses (A Court of Thorns...,Sarah J. Maas,761,"4.18&3,068,237 & 2015",https://www.goodreads.com/book/show/50659467-a...,https://i.gr-assets.com/images/S/compressed.ph...,4.18,"3,068,237",2015


In [51]:
booktok_books_df_final = booktok_books_df_final.drop('Meta', axis=1)
booktok_books_df_final

,Title,Author,Num_shelved_booktok,Goodreads URL,Image URL,avg rating,num_ratings,year_published
0,The Seven Husbands of Evelyn Hugo (Hardcover),Taylor Jenkins Reid,1023,https://www.goodreads.com/book/show/32620332-t...,https://i.gr-assets.com/images/S/compressed.ph...,4.41,"3,340,727",2017
1,It Ends with Us (It Ends with Us #1),Colleen Hoover,875,https://www.goodreads.com/book/show/27362503-i...,https://i.gr-assets.com/images/S/compressed.ph...,4.14,"3,863,992",2016
2,The Song of Achilles (Paperback),Madeline Miller,806,https://www.goodreads.com/book/show/13623848-t...,https://i.gr-assets.com/images/S/compressed.ph...,4.32,"1,628,438",2011
3,"The Cruel Prince (The Folk of the Air, #1)",Holly Black,783,https://www.goodreads.com/book/show/26032825-t...,https://i.gr-assets.com/images/S/compressed.ph...,4.03,"1,378,006",2018
4,A Court of Thorns and Roses (A Court of Thorns...,Sarah J. Maas,761,https://www.goodreads.com/book/show/50659467-a...,https://i.gr-assets.com/images/S/compressed.ph...,4.18,"3,068,237",2015
...,...,...,...,...,...,...,...,...
1245,"Barbarian Lover (Ice Planet Barbarians, #3)",Ruby Dixon,19,https://www.goodreads.com/book/show/60629205-b...,https://i.gr-assets.com/images/S/compressed.ph...,3.84,"62,683",2015
1246,When Sinners Play (Sinners of Hawthorne Univer...,Eva Ashwood,19,https://www.goodreads.com/book/show/61095459-w...,https://i.gr-assets.com/images/S/compressed.ph...,4.03,"8,489",2020
1247,Cultish: The Language of Fanaticism (Hardcover),Amanda Montell,19,https://www.goodreads.com/book/show/55338982-c...,https://i.gr-assets.com/images/S/compressed.ph...,3.85,"91,989",2021
1248,The Memory Police (Hardcover),Yōko Ogawa,19,https://www.goodreads.com/book/show/37004370-t...,https://i.gr-assets.com/images/S/compressed.ph...,3.72,"100,268",1994


In [61]:
driver.quit()

In [63]:
#save df as a csv 

booktok_books_df_final.to_csv('booktok_books_df_final2.csv', index=False)

OH MY GOD I GOT MY DATA!!!!!!!! i was just mixing the requests and the selenium tools this whole time :(